## Day 14

https://adventofcode.com/2021/day/14

In [1]:
def readInput14(filename):
    with open(filename) as f:
        t = f.read().split("\n\n")
        rules = {}
        for r in t[1].strip('\n').split("\n"):
            rr = r.split(" -> ")
            rules[rr[0]] = rr[1]
        return t[0], rules

template0, rules0 = readInput14("./data/day14test1.txt")
template , rules  = readInput14("./data/input14.txt")

### Part 1

In [2]:
def stepString(tt_,rules):
    tt = list(tt_)
    return "".join(step(tt,rules))

def step(tt,rules):
    ttnew = [ tt[0] ]
    for i in range(len(tt)-1):
        pair = tt[i]+tt[i+1]
        if pair in rules.keys():
            ttnew.append(rules[pair])
        ttnew.append(tt[i+1])
    return(ttnew)

def evolve(template,rules,nstep,verbose=True):
    if verbose:
        print("Template:    ",template)
    tt = list(template)
    for i in range(nstep):
        tt = step(tt,rules)
        if verbose:
            print("After step {}: {}".format(i+1,"".join(tt)))
    return tt

In [3]:
evolve(template0,rules0,nstep=4,verbose=True)

from collections import Counter

ttlast0 = evolve(template0,rules0,nstep=10,verbose=False)
count0 = Counter(ttlast0)
print("Test 1:",max(count0.values())-min(count0.values()))

Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
Test 1: 1588


In [4]:
ttlast = evolve(template,rules,nstep=10,verbose=False)
count = Counter(ttlast)
print(max(count.values())-min(count.values()))

2003


### Polymer as a Linked List

In [5]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

    def __repr__(self):
        return self.data

class LinkedList:
    def __init__(self, nodes=None):
        self.head = None
        self.tail = None
        if nodes is not None:
            data=nodes.pop(0)
            node = Node(data)
            self.head = node
            for elem in nodes:
                node.next = Node(data=elem)
                node = node.next
            self.tail = node

    def __repr__(self):
        node = self.head
        nodes = []
        while node is not None:
            nodes.append(node.data)
            node = node.next
        return "".join(nodes)
    
    def __iter__(self):
        node = self.head
        while node is not None:
            yield node
            node = node.next

In [6]:
def stepLinked(polymer,rules):
    p = polymer.head
    while p.next:
        last = p.next
        pair = p.data+last.data
        if pair in rules.keys():
            insert = Node(rules[pair])
            insert.next = p.next
            p.next = insert
        p = last 

def evolveLinked(template,rules,nstep,verbose=True):
    if verbose:
        print("Template:    ",template)
    polymer = LinkedList(list(template))
    for i in range(nstep):
        stepLinked(polymer,rules)
        if verbose:
            print("After step {}: {}".format(i+1,polymer))
    return polymer

from collections import defaultdict

def countElement(polymer):
    counter = defaultdict(lambda: 0)
    for p in polymer:
        counter[p.data] += 1
    return counter

In [7]:
polymer_final0 = evolveLinked(template0,rules0,nstep=4)

Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB


In [8]:
polymer_final0 = evolveLinked(template0,rules0,nstep=10,verbose=False)
c = countElement(polymer_final0)
print("Test 1:",max(c.values())-min(c.values()))

Test 1: 1588


In [9]:
polymer_final = evolveLinked(template,rules,nstep=10,verbose=False)
c = countElement(polymer_final)
print("Part 1:",max(c.values())-min(c.values()))

Part 1: 2003


### Part 2

Similarly to Day 6, better keep track of the polymer and polymer pair frequencies instead of evolving the full string, since its lenght increases exponentially!

In [10]:
from collections import defaultdict

def evolveQuick(template,rules,nsteps):
    # Count polymers in template
    freqs = defaultdict(int)
    for p in template:
        freqs[p] += 1

    # Count polymer pairs in template
    pairs = defaultdict(int)    
    for i in range(len(template)-1):
        pairs[template[i:i+2]] += 1

    # Evolve and track pairs and letter frequencies
    for n in range(nsteps):
        pairsnew = defaultdict(int)
        for p in pairs.keys():
            # new pairs from polymer insertion
            p1 = p[0]+rules[p]
            p2 = rules[p]+p[1]
            pairsnew[p1] += pairs[p]
            pairsnew[p2] += pairs[p]
            # add letter from polymer insertion
            freqs[rules[p]] += pairs[p]
        pairs = pairsnew

    return freqs

In [11]:
freqs0 = evolveQuick(template0,rules0,nsteps=10)
print("Test 1:",max(freqs0.values())-min(freqs0.values()))

freqs = evolveQuick(template,rules,nsteps=10)
print("Part 1:",max(freqs.values())-min(freqs.values()))

Test 1: 1588
Part 1: 2003


In [12]:
freqs0 = evolveQuick(template0,rules0,nsteps=40)
print("Test 2:",max(freqs0.values())-min(freqs0.values()))

freqs = evolveQuick(template,rules,nsteps=40)
print("Part 2:",max(freqs.values())-min(freqs.values()))

Test 2: 2188189693529
Part 2: 2276644000111
